In [58]:
import os
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [70]:
names = os.listdir('./경남')

In [71]:
names = [name[:-4] for name in names]

In [ ]:
names

In [74]:
df_경상남도 = pd.DataFrame()

for name in names:
    df = pd.read_csv(f'./경상남도/{name}.csv',encoding='ANSI')
    df_경상남도 = pd.concat([df_경상남도,df],ignore_index=True)

==================================================================================================

==================================================================================================

==================================================================================================

In [ ]:
# 파일명 변경

folder_names = os.listdir('./경남/')

for folder_name in folder_names:
    file_names = os.listdir(f'./경남/{folder_name}')
    for file_name in file_names:
        split_file_name = file_name.replace('.csv','').split('_')
        os.rename(f'./경남/{folder_name}/{file_name}',f'./경남/{folder_name}/{folder_name}_{split_file_name[1]}.csv')

==================================================================================================

==================================================================================================

==================================================================================================

### 데이터프레임 합치기
### 관광지 기본정보

In [ ]:
basic_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_관광지 기본정보.csv',encoding='ANSI')
    basic_info = pd.concat([basic_info,df],ignore_index=True)

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기

### 내비게이션

In [ ]:
navi_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_내비게이션 목적지 검색량 추이.csv',encoding='ANSI',index_col=0)
    df = df.rename(columns={df.columns[0] : folder_name}).transpose()
    df['관광지명'] = folder_name
    navi_info = pd.concat([navi_info,df])

In [ ]:
order = ['관광지명',202301,202302, 202303, 202304,202305, 202306, 202307, 202308,202309, 202310, 202311, 202312]
navi_info = navi_info[order]

In [ ]:
navi_info.to_csv('./경남/navi_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 방문자 거주지별 분포

In [ ]:
order = ['관광지명','광역지자체명', '기초지자체명', '광역지자체별 거주 방문자 비율', '기초지자체별 거주 방문자 비율']

visit_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_방문자 거주지별 분포.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    
    visit_info = pd.concat([visit_info,df],ignore_index=True)

In [ ]:
visit_info.to_csv('./경남/visit_info.csv',encoding='utf-8',index=False)

### 데이터프레임 합치기
### 성별

In [ ]:
order = ['관광지명','연령대', '남성방문 비율', '여성방문 비율']

gender_info = pd.DataFrame()

for folder_name in folder_names:
    df = pd.read_csv(f'./경남/{folder_name}/{folder_name}_성·연령별 방문자.csv',encoding='ANSI')
    df['관광지명'] = folder_name
    df = df[order]
    gender_info = pd.concat([gender_info,df],ignore_index=True)

In [ ]:
gender_info.to_csv('./경남/gender_info.csv',encoding='utf-8',index=False)

### 데이터 가져오기
### 위치정보

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
url = 'https://datalab.visitkorea.or.kr/visualize/getTempleteData.do'
city_code = [48125,48127,48123,48121,48129]

all = []

for i in city_code:
    params = {
        "SIDO_CD": 29,
        "SGG_CD": i,
        "BASE_YM1": 2010,
        "BASE_YM2": 2022,
        "CONT_NM": "",
        "KTO_CATE_LCLS_CD": "",
        "KTO_CATE_MCLS_CD": "",
        "firstIndex": float('nan'),
        "pageIndex": 1,
        "recordCountPerPage": 10,
        "orderSelect": "",
        "orderValue": "asc",
        "contentList": "전체",
        "noTheme": "",
        "qid": "LN_05_01_011"
    }

    response = requests.get(url,params=params)
    data = response.json()
    
    for item in data.get('list', []):
        result = {
            '관광지명': item.get('CONT_NM'),
            '주소': item.get('TAR_ADDR_NM'),
            '대분류': item.get('KTO_CATE_LCLS_NM'),
            '중분류': item.get('KTO_CATE_MCLS_NM'),
            '소분류': item.get('KTO_CATE_SCLS_NM'),
            'longitude': item.get('XCRD'),
            'latitude': item.get('YCRD')
        }
        all.append(result)
        
    if len(all) == 101:
        break

In [ ]:
df = pd.DataFrame(all)

In [ ]:
df = pd.merge(basic_info,df,on='관광지명',how='outer')

In [ ]:
df.to_csv('./경남/location_info.csv',encoding='utf-8',index=False)

==================================================================================================

==================================================================================================

==================================================================================================

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

stars = []
reviews = []

for i in basic_info['관광지명']:
    url = f'https://www.google.co.kr/maps/search/{i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    
    try:
        # 해당 클래스명이 나타날 때까지 최대 10초 대기
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'F7nice'))
        )
        
        star = element.text.split('\n')[0]
        review = element.text.split('\n')[1][1:-1]
        
        stars.append(star)
        reviews.append(review)
    except:
        check = input('일치하는 장소를 클릭 후 y를 눌러주세요(이외 None)')
        if check.upper() == 'Y':
            star = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[0]
            review = driver.find_element(By.CLASS_NAME, 'F7nice').text.split('\n')[1][1:-1]
            
            stars.append(star)
            reviews.append(review)
        else:
            stars.append(None)
            reviews.append(None)
    
    driver.quit()
    print(f'평점 수 : {len(stars)}','|',f'리뷰 수 : {len(reviews)}','|',f'관광지 명 : {i}','|', f'별점 : {star}','|',f'리뷰 : {review}')
    
    star = None
    review = None

In [ ]:
print(f'총 데이터 수 : {len(basic_info)}')
print('=========================================')
print(f'평점 수 : {len(stars)}')
print(f'리뷰 수 : {len(reviews)}')
print('=========================================')
print(f'총 None값 수 : {stars.count(None)}')
print(f'총 None값 수 : {reviews.count(None)}')

총 데이터 수 : 49
평점 수 : 49
리뷰 수 : 49
총 None값 수 : 2
총 None값 수 : 2


In [ ]:
basic_info['stars'] = stars
basic_info['reviews'] = reviews

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

### 리뷰, 체류시간 -> 숫자 값으로 변경

In [ ]:
time = []

for i in basic_info['평균 체류시간']:
    i = i.replace('시간','').replace('분','').strip()
    i = i.split(' ')
    time.append((int(i[0]) * 60) + int(i[1]))
    
basic_info['평균 체류시간'] = time

In [ ]:
reviews_to_int = []

for i in basic_info['reviews']:
    if type(i) == type(None):
        reviews_to_int.append(i)
        continue
    if type(i) in (float,int):
        reviews_to_int.append(i)
    else:
        i = i.replace(',','')
        i = int(i)
        reviews_to_int.append(i)

basic_info['reviews'] = reviews_to_int

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

### None값 -> 관광지별 평균

In [ ]:
navi_info.set_index('관광지명',inplace=True)

In [ ]:
navi_info = navi_info.apply(lambda row: row.fillna(int(row.mean())), axis=1)

In [ ]:
navi_info.to_csv('./경남/navi_info.csv',encoding='utf-8',index=False)

### 광역지자체별 거주 방문자 비율

In [ ]:
visit_info.dropna(subset=['광역지자체명'], axis=0,inplace=True)

In [ ]:
names = visit_info['관광지명'].tolist()

local_visit = visit_info[visit_info['광역지자체명'].str.contains('광주')].groupby('관광지명')[['광역지자체별 거주 방문자 비율']].mean()
local_visit = local_visit.reset_index()

local_visit.loc[local_visit['관광지명'].isin(names)]

In [ ]:
basic_info['광역지자체별 거주 방문자 비율'] = local_visit['광역지자체별 거주 방문자 비율']

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

search_number = []
for i in basic_info['관광지명']:
    if not i.startswith('창원'):
        i = '창원 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    search_number.append(driver.find_element(By.ID, 'result-stats').text)

basic_info['search_number'] = search_number

In [ ]:
search_number = []
for index,i in enumerate(basic_info['관광지명']):
    if basic_info.iloc[index]['search_number']:
        continue
    if not i.startswith('창원'):
        i = '창원 ' + i
    url = f'https://www.google.com/search?q={i}'
    service = Service(executable_path='./data/chromedriver.exe')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    time.sleep(2)

    # search_number.append(driver.find_element(By.ID, 'result-stats').text)
    basic_info.loc[index,'search_number'] = driver.find_element(By.ID, 'result-stats').text

In [ ]:
import re

pattern = re.compile(r'\([^)]*\)')

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: re.sub(pattern,'',x).strip())

basic_info['search_number'] = basic_info['search_number'].apply(lambda x: int(x.replace('개','').replace(',','')[7:]))

In [ ]:
basic_info.to_csv('./경남/basic_info.csv',encoding='utf-8',index=False)

===============================================================================================================

===============================================================================================================

===============================================================================================================

In [3]:
import pandas as pd

basic_info = pd.read_csv('./경남/basic_info.csv')
gender_info = pd.read_csv('./경남/gender_info.csv')
location_info = pd.read_csv('./경남/location_info.csv')
navi_info = pd.read_csv('./경남/navi_info.csv')
visit_info = pd.read_csv('./경남/visit_info.csv')

In [4]:
totals = []
avgs = []

for i in range(len(navi_info)):
    total = navi_info.iloc[i][1:].sum()
    avg = navi_info.iloc[i][1:].mean()
    
    totals.append(total)
    avgs.append(avg)
navi_info['sum'] = totals
navi_info['avg'] = avgs

In [5]:
df = pd.DataFrame()
df = pd.concat([basic_info,navi_info[['sum','avg']]],axis=1)
df = pd.merge(df,location_info,on='관광지명',how='inner')

In [6]:
import numpy as np
score = []

for i in range(len(df)):
    star = df.iloc[i]['stars'] * 1.2
    review = np.log1p(max(df['reviews'])) - np.log1p(df.iloc[i]['reviews']) * 1.1
    search = np.log1p(max(df['search_number'])) - np.log1p(df.iloc[i]['search_number'])
    if pd.isna(df.iloc[0]['광역지자체별 거주 방문자 비율']):
        visit = 0
    else:
        visit = np.log1p(df.iloc[i]['광역지자체별 거주 방문자 비율']) * 1.1
    time = np.log1p(df.iloc[i]['평균 체류시간']) * 0.8
    navi = np.log1p(max(df['avg'])) - np.log1p(df.iloc[i]['avg'])
    all = star+review+search+visit+time-navi
    if df.iloc[i]['소분류'] == '산':
        score.append(all*0.9)
    elif df.iloc[i]['소분류'] == '사찰':
        score.append(all*0.9)
    else:
        score.append(all)

df['scores'] = score

In [8]:
first = df.sort_values(by='scores',ascending=False)

In [13]:
import folium

gyeongnam_map = folium.Map(location=[35.2279728,128.681882],zoom_start=11)

In [14]:
for i in range(len(first)):
    if i == 30:
        break
    folium.Marker(
            [first.iloc[i]['latitude'],first.iloc[i]['longitude']],
            popup=folium.Popup(first.iloc[i]['관광지명'],maxWidth=300),
            fill_opacity=0.5,
            icon=folium.Icon(color='blue')            
            ).add_to(gyeongnam_map)

In [15]:
gyeongnam_map